# Obs and LDR meta-modeling

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Make the graphs a bit prettier, and bigger
# pd.set_option('display.mpl_style', 'default') 
# pd.set_option('display.width', 5000) 
# pd.set_option('display.max_columns', 60) 

In [3]:
%matplotlib inline

In [4]:
from sklearn import linear_model

In [5]:
from scipy import stats, integrate

In [6]:
import hillmaker as hm
from pandas import Timestamp
import re
from datetime import datetime

In [ ]:
## Read training data set
train_df = pd.read_csv('train_exp9_tandem05_nodischadj.csv')
train_df.info()

In [ ]:
train_df.head()

Let's start by focusing on the OBS and LDR units. Since this part of the network uses recovery blocking, we can explore approximations for such systems. Also, from a practical point of view, getting blocked in OBS is an undesirable occurance. We can also do some basic sanity checking since OBS was modeled as (essentially) infinite capacity and exponential service. Thus, we have an $M/M/\infty$ queue feeding an $M/E2/c_L$ queue. Furthermore, Scenarios 136-150 are infinite capacity: $M/M/\infty --> M/E2/\infty$.

## OBS unit occupancy for infinite capacity scenarios
Since $M/G/\infty$ systems have Poisson occupancy, we can create a histogram of occupancy for one or more of the infinite capacity scenarios. The log files are at OU right now. However, we can at least look at means and variances of occupancy since they should be equal for Poisson distributions. Oops, hmout files are at OU too and like an idiot I didn't put occupancy sd's or variances into the results file. Doh!

In [ ]:
file_stopdata = './unit_stop_log_Experiment1_Scenario138_Rep17.csv'
stops_df = pd.read_csv(file_stopdata)
stops_df.info() # Check out the structure of the resulting DataFrame

rx = re.compile(r'Scenario([0-9]{1,3})_Rep([0-9]{1,2})')

m = re.search(rx, file_stopdata)
scenario_name = m.group(0)
scenario_num = m.group(1)
rep_num = m.group(2)
print (scenario_name, scenario_num, rep_num)

In [ ]:
stops_df.head()

In [ ]:
in_fld_name = 'EnteredTS'
out_fld_name = 'ExitedTS'
cat_fld_name = 'Unit'
start_analysis = '12/12/2015 00:00'
end_analysis = '12/19/2016 00:00'

# Optional inputs

tot_fld_name = 'OBTot'
bin_size_mins = 1
#includecats = ['Obs', 'LDR', 'PP']
outputpath = '.'

In [ ]:
basedate = Timestamp('20150215 00:00:00')
stops_df['EnteredTS'] = stops_df.apply(lambda row:
                                   Timestamp(round((basedate + pd.DateOffset(hours=row['Entered'])).value,-9)), axis=1)


stops_df['ExitedTS'] = stops_df.apply(lambda row:
                                  Timestamp(round((basedate + pd.DateOffset(hours=row['Exited'])).value,-9)), axis=1)

# Filter stops data by start and end analysis dates
start_analysis_dt = pd.Timestamp(start_analysis)
end_analysis_dt = pd.Timestamp(end_analysis)

stops_df = stops_df[(stops_df['EnteredTS'] <= end_analysis_dt) & (stops_df['ExitedTS'] >= start_analysis_dt)]



In [ ]:
# Create bydate table
bydt = hm.bydatetime.make_bydatetime(stops_df,in_fld_name, out_fld_name,
                         start_analysis,end_analysis,cat_fld_name,
                         total_str=tot_fld_name,bin_size_minutes=bin_size_mins,
                         edge_bins=2)

In [ ]:
bydt.head()

In [ ]:
bydt.to_csv("bydt_minute.csv")

In [ ]:
obs_occ = bydt.ix[bydt.category=='Obs','occupancy']
ldr_occ = bydt.ix[bydt.category=='LDR','occupancy']
pp_occ = bydt.ix[bydt.category=='PP','occupancy']

obs_occ.to_csv("obs_occ.csv")
ldr_occ.to_csv("ldr_occ.csv")
pp_occ.to_csv("pp_occ.csv")

In [ ]:
obs_occ.describe()

In [7]:
obs_occ = pd.read_csv("obs_occ.csv",header=None)
ldr_occ = pd.read_csv("ldr_occ.csv",header=None)
pp_occ = pd.read_csv("pp_occ.csv",header=None)

In [ ]:
obs_occ.head()

In [8]:
obs_occ[2].describe()

count    537121.000000
mean          0.266858
std           0.526885
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           5.000000
Name: 2, dtype: float64

In [9]:
ldr_occ[2].describe()

count    537121.000000
mean          1.373270
std           1.202208
min           0.000000
25%           0.000000
50%           1.000000
75%           2.000000
max           8.000000
Name: 2, dtype: float64

In [12]:
ldr_occ[2].var()

1.4453041825604998

In [10]:
pp_occ[2].describe()

count    537121.000000
mean          6.278457
std           2.739540
min           0.000000
25%           4.000000
50%           6.000000
75%           8.000000
max          18.000000
Name: 2, dtype: float64

In [11]:
pp_occ[2].var()

7.505081931933824

In [ ]:
sns.distplot(pp_occ[2], kde=False, fit=stats.poisson)

In [ ]:
obs_occ[(obs_occ > 0) & (obs_occ < 1)].head()

In [ ]:
obs_occ.hist()

In [ ]:
ldr_occ.hist()

In [ ]:
pp_occ.hist()

In [ ]:
# Run hillmaker
hills = hm.make_hills(scenario_name,stops_df,in_fld_name, out_fld_name,
                         start_analysis,end_analysis,cat_fld_name,
                         total_str=tot_fld_name,bin_size_minutes=bin_size_mins,
                         return_dataframes=True, export_path=outputpath)

In [ ]:
hills.keys()

In [ ]:
bydt = hills['bydatetime']

In [ ]:
sns.distplot(bydt.ix[bydt.category=='LDR','occupancy'])

In [ ]:
bydt.ix[bydt.category=='LDR','occupancy'].describe()

In [ ]:
sns.distplot(bydt.ix[bydt.category=='PP','occupancy'])

In [ ]:
bydt.ix[bydt.category=='PP','occupancy'].describe()

In [ ]:
2.343 ** 2

So, why aren't the occupancy stats suggestive of Pn distribution? Shouldn't mean occupancy = variance of occupancy? Does this have something to do with the way partial occupancy stats are done in hillmaker?

## OBS load vs occupancy scatter
I already did these in R, but let's do them in Python using `matplotlib`, `seaborn`, and even y-hat's `ggplot` and, why not, `ggplot2` via RMagic.

In [ ]:
train_infcap = train_df[train_df.scenario >= 136]

In [ ]:
train_infcap.head()

## Matplotlib and Seaborn

In [ ]:
# Matplotlib
plt.scatter(train_infcap.load_obs,train_infcap.occ_mean_mean_obs)

In [ ]:
#Seaborn
sns.regplot("load_obs","occ_mean_mean_obs",data=train_infcap)

In [ ]:
plt.scatter(train_infcap.load_ldr,train_infcap.occ_mean_mean_ldr)

In [ ]:
sns.regplot("load_ldr","occ_mean_mean_ldr",data=train_infcap)

## y-hat's ggplot and ggplot2 via RMagic
Right now there seems to be issues with rpy2 on Python 3.5. Not sure about using Conda to install ggplot as it's not included and not sure about the stuff on anaconda.org. I could pip install it.

http://eneskemalergin.github.io/2015/10/01/R_Magic_with_IPython/

https://bitbucket.org/rpy2/rpy2/issues/313/typeerror-type-rpy2rinterfacestrsexpvector

In [ ]:
# Load the extension - PROBLEMS RIGHT NOW WITH RPY2 ON PYTHON 3.5
import rpy2.robjects.lib.ggplot2 as ggplot2
%load_ext rpy2.ipython

In [ ]:
%Rpush train_infcap
%R str(train_infcap)

ggplot(data=results_infcap) + aes(x=load_obs, y=occ_mean_mean_obs) + geom_point() + geom_abline()
ggplot(data=results_infcap) + aes(x=load_ldr, y=occ_mean_mean_ldr) + geom_point() + geom_abline()
ggplot(data=results_infcap) + aes(x=load_pp, y=occ_mean_mean_pp) + geom_point() + geom_abline()

In [ ]:
from ggplot import *

In [ ]:
# y-hat's ggplot

ggplot(aes(x='load_ldr', y='occ_mean_mean_ldr'), data=train_infcap) + geom_point() + geom_abline()

# Using my qng library

In [ ]:
# For now I'm just sticking qng.py into this directory
import qng

In [ ]:
barber_arr = 1.37
barber_svc = 1
barber_c = 4

In [ ]:
qng.mgc_prob_wait_erlangc(barber_arr, barber_svc, barber_c)

In [ ]:
help(qng.mgc_prob_wait_erlangc)

In [ ]:
prob_wait_ldr = [qng.mgc_prob_wait_erlangc(a,b,c) for (a,b,c) in zip(train_df.lam_ldr,1/train_df.alos_ldr,train_df.cap_ldr)]

In [ ]:
plt.scatter(train_df.mean_pct_waitq_ldr,prob_wait_ldr)

In [ ]:
prob_waitq_ldr_df = pd.DataFrame(dict(mean_pct_waitq_ldr=train_df.mean_pct_waitq_ldr,
                                      prob_wait_ldr=prob_wait_ldr,
                                      cap_ldr=train_df.cap_ldr,
                                      cap_pp=train_df.cap_pp),index=None)

In [ ]:
prob_waitq_ldr_df[100:120]

In [ ]:
prob_waitq_ldr_df.head(20)

The short listing above shows the challenge. Each group of three scenarios have the same LDR capacity and birth volume and thus have the same value for predicted prob_wait_ldr based on a simple ErlangC approximation. However, the actual percentage that wait differs for the first three scenarios due to blocking in LDR for different PP capacity levels in the first three scenarios. This is good in the sense that it shows the problem is non-trivial.

In [ ]:
help(qng.erlangc)

In [ ]:
.2 * 72 + .8 * 48

In [ ]:
train_df.head(20)['actual_los_mean_mean_pp']

Looking at the actual LDR alos values, we see that some are below 12 and some are higher than 12. One would think that the scenarios for which actual > 12 correspond to significant blocking in LDR and little queueing to get into LDR (since LDR LOS is reduced by queueing time).

In [ ]:
train_df.head(10)['actual_los_mean_mean_ldr']

If we use the actual mean LDR alos, how does erlangC do in predicting probability of waiting in obs to get into an LDR bed? 

In [ ]:
prob_wait_ldr_approx = [qng.mgc_prob_wait_erlangc(a,b,c) for (a,b,c) 
                        in zip(train_df.lam_ldr,1/(train_df.actual_los_mean_mean_ldr/24.0),train_df.cap_ldr)]

In [ ]:
plt.scatter(train_df.mean_pct_waitq_ldr,prob_wait_ldr_approx)

Clearly, it performs quite well. So, need a good estimate of actual ALOS in LDR. That's tricky.

In [ ]:
plt.scatter(train_df.alos_ldr,train_df.actual_los_mean_mean_ldr/24.0)

In [ ]:
plt.scatter(train_df.rho_pp,train_df.actual_los_mean_mean_ldr/24.0,c=train_df.rho_ldr)
plt.legend()

In [ ]:
sns.lmplot(x='rho_pp',y='actual_los_mean_mean_ldr',data=train_df,hue='rho_ldr',fit_reg=False)

Is actual ALOS in LDR equal to planned LOS + average block time - average queue time? Don't have to worry about the discharge timing offset since this set of runs didn't do any adjustments for discharge timing.

In [ ]:
train_df['actual_los_mean_est_ldr'] = train_df['alos_ldr']*24.0 + \
    train_df['mean_blocked_by_pp_mean'] * train_df['mean_pct_blocked_by_pp'] * (1-train_df['tot_c_rate'])- \
    train_df['mean_waitq_ldr_mean'] * train_df['mean_pct_waitq_ldr']

In [ ]:
sns.lmplot(y='actual_los_mean_mean_ldr',x='actual_los_mean_est_ldr',data=train_df)

# Postpartum
Let's explore PP unit a bit. First of all, if we did an discharge timing adjustment, need to know how that changed the overall ALOS in PP. Feels like we could compute the bias from the discharge timing distribution as the difference from the middle of the day (t=12). For this set of runs with no discharge adjustment, actual and planned los should be the same.

In [ ]:
alos_pp_lm1 = linear_model.LinearRegression()

In [ ]:
X = train_df['planned_los_mean_mean_pp']

In [ ]:
y = train_df['actual_los_mean_mean_pp']

In [ ]:
alos_pp_lm1.fit(X.reshape(-1,1),y)

In [ ]:
print(alos_pp_lm1.intercept_,alos_pp_lm1.coef_)

In [ ]:
sns.lmplot(y='actual_los_mean_mean_pp',x='planned_los_mean_mean_pp',data=train_df)

## Blocking probability

In [ ]:
params = [(a,b,c) for (a,b,c) 
                        in zip(train_df.lam_pp,1.0/(train_df.alos_pp),train_df.cap_pp)]

In [ ]:
params[0]

In [ ]:
qng.mgc_prob_wait_erlangc(*params[0])

In [ ]:
prob_blocked_by_pp_approx = [qng.mgc_prob_wait_erlangc(a,b,c) for (a,b,c) 
                        in zip(train_df.lam_pp,1.0/(train_df.alos_pp),train_df.cap_pp)]

In [ ]:
prob_blocked_by_pp_approx[0:10]

In [ ]:
train_df['mean_pct_blocked_by_pp'][0:10]

In [ ]:
train_df['prob_blocked_by_pp_approx'] = pd.Series(prob_blocked_by_pp_approx)

In [ ]:
sns.lmplot(y='mean_pct_blocked_by_pp',x='prob_blocked_by_pp_approx',data=train_df)

In [ ]:
sns.distplot(train_df['mean_pct_blocked_by_pp'], kde=True, rug=True);

In [ ]:
sns.distplot(train_df['prob_blocked_by_pp_approx'], kde=True, rug=True);

Hmm, why are these plots so different?